In [2]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd

# 노래제목 크롤링하기

In [ ]:
years = ['2022', '2021', '2020']
gnrs = {
    'GN0100': '발라드',
    'GN0200': '댄스',
    'GN0300': '힙합',
    'GN0700': '트로트',
    'GN0800': '포크',
    'GN2400': '국악',
  }
df = pd.DataFrame(columns=['genre', 'song_id', 'artist_id', 'song_name', 'artist_name'])

for year in years:
  months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

  # 2022년은 6월까지만 있음
  if(year == '2022'):
    months = ['01', '02', '03', '04', '05', '06']

  for month in months:
    # 장르별 클릭
    for gnrid, gnrname in gnrs.items():
      # random time
      time.sleep(random.randint(1, 4))

      url = 'https://www.melon.com/chart/search/list.htm?chartType=MO&age=2020&year='+year+'&mon='+month+'&classCd='+gnrid+'&moved=Y'
      # set headers
      headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36',
        'Referer': 'https://www.melon.com/chart/search/index.htm',
        'Host': 'www.melon.com',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Dest': 'empty',
        'Accept': '*/*',
        'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
        'X-Requested-With': 'XMLHttpRequest',
        'Connection': 'keep-alive',
        'Cookie':'__T_=1; PCID=16588238875659253194716; PC_PCID=16588238875659253194716; POC=MP10; _T_ANO=MV0k3Iq2XCCiPOOi7xfq0iLAARL/I91rWBM5GGZ9C/FRgaBUoFDBzSqHGq56Wbm100ztNv4VOQIOZV6CjdPDjqh3a6Oetqo4nAeL8UUDs6AceTzaxt8/RbNboG3G4uK0wvYq+aQi2jpN5Ztr2XpkdLIuLMjElj/9RQpNE1wP/Y/sEVbRVnTKI4d3aKWEz5Q3W2fF0F1M4eTl9vFYfdOxbF9SmwAx2A2rLG6rJRPyhC5fpRB55bex0knivxhcpF2T3TdhF6A+28g2AQOTWmfxN1Ov7BzL97c6XEjy6wD0dGJ9yVmkcu+a+Usmv2aavK/p+0fmCd1b1mG3+D1JiHoQGw==; wcs_bt=s_f9c4bde066b:1658884565'

      }
      # request
      response = requests.get(url, headers=headers)
      # print(response.status_code)
      # html
      html = response.text
      # soup
      soup = BeautifulSoup(html, 'html.parser')

      songs = soup.select('.wrap_song_info')

      for song in songs:
        song_info = song.select_one('.rank01 a:first-child')
        if(song_info is not None):
          song_id = song_info['href']
        else:
          continue

        # song id not None
        if(song_id != None):
          song_id = song_id.split("'")[-2]
        else:
          continue
        
        song_name = song_info.text

        artist_info = song.select_one('.rank02 a:first-child')
        if(artist_info is not None):
          artist_id = artist_info['href']
        else:
          continue

        # artist id not None
        if(artist_id != None):
          artist_id = artist_id.split("'")[-2]
        else:
          continue

        artist_name = artist_info.text

        df = df.append({'genre': gnrname, 'song_id': song_id, 'artist_id': artist_id, 'song_name': song_name, 'artist_name': artist_name}, ignore_index=True)
        

df.to_csv('new_new.csv', index=False)

In [5]:
# 중복제거 등
df = pd.read_csv('new_new.csv')

df['song_name_artistid'] = ''

for i in range(len(df)):
  songname = str(df.iloc[i, 3])
  songname = songname.split('(')[0]

  df.iloc[i, 5] = songname + '_' + str(df.iloc[i, 4])

# drop if 'song_name' column string include 'inst'
df = df[~df['song_name'].str.lower().str.contains('inst')]
df = df[~df['song_name'].str.lower().str.contains('japanese ver')]
df = df[~df['song_name'].str.lower().str.contains('chinese ver')]
df = df[~df['song_name'].str.lower().str.contains('english ver')]
df = df[~df['song_name'].str.lower().str.contains('remix')]

df = df.drop_duplicates(subset=['song_name_artistid'])
df = df.reset_index(drop=True)

# drop 'song_name_artistid' column
df = df.drop(columns=['song_name_artistid'])

# save df to csv
df.to_csv('songs_check.csv', index=False)

In [8]:
songs = pd.read_csv('songs_check.csv')
songs.head()

# count songs group by genre
songs['genre'].value_counts()

발라드    557
힙합     473
댄스     443
트로트    425
국악     312
포크     275
Name: genre, dtype: int64

# 포크 국악 보충

In [9]:
years = ['2019', '2018']
gnrs = {
    'GN0800': '포크',
    'GN2400': '국악',
  }
df = pd.DataFrame(columns=['genre', 'song_id', 'artist_id', 'song_name', 'artist_name'])

for year in years:
  months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

  for month in months:
    # 장르별 클릭
    for gnrid, gnrname in gnrs.items():
      # random time
      time.sleep(random.randint(1, 4))

      url = 'https://www.melon.com/chart/search/list.htm?chartType=MO&age=2010&year='+year+'&mon='+month+'&classCd='+gnrid+'&moved=Y'
      # set headers
      headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36',
        'Referer': 'https://www.melon.com/chart/search/index.htm',
        'Host': 'www.melon.com',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Dest': 'empty',
        'Accept': '*/*',
        'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
        'X-Requested-With': 'XMLHttpRequest',
        'Connection': 'keep-alive',
        'Cookie':'__T_=1; PCID=16588238875659253194716; PC_PCID=16588238875659253194716; POC=MP10; _T_ANO=MV0k3Iq2XCCiPOOi7xfq0iLAARL/I91rWBM5GGZ9C/FRgaBUoFDBzSqHGq56Wbm100ztNv4VOQIOZV6CjdPDjqh3a6Oetqo4nAeL8UUDs6AceTzaxt8/RbNboG3G4uK0wvYq+aQi2jpN5Ztr2XpkdLIuLMjElj/9RQpNE1wP/Y/sEVbRVnTKI4d3aKWEz5Q3W2fF0F1M4eTl9vFYfdOxbF9SmwAx2A2rLG6rJRPyhC5fpRB55bex0knivxhcpF2T3TdhF6A+28g2AQOTWmfxN1Ov7BzL97c6XEjy6wD0dGJ9yVmkcu+a+Usmv2aavK/p+0fmCd1b1mG3+D1JiHoQGw==; wcs_bt=s_f9c4bde066b:1658884565'

      }
      # request
      response = requests.get(url, headers=headers)
      print(response.status_code)
      # html
      html = response.text
      # soup
      soup = BeautifulSoup(html, 'html.parser')

      songs = soup.select('.wrap_song_info')

      for song in songs:
        song_info = song.select_one('.rank01 a:first-child')
        if(song_info is not None):
          song_id = song_info['href']
        else:
          continue

        # song id not None
        if(song_id != None):
          song_id = song_id.split("'")[-2]
        else:
          continue
        
        song_name = song_info.text

        artist_info = song.select_one('.rank02 a:first-child')
        if(artist_info is not None):
          artist_id = artist_info['href']
        else:
          continue

        # artist id not None
        if(artist_id != None):
          artist_id = artist_id.split("'")[-2]
        else:
          continue

        artist_name = artist_info.text

        df = df.append({'genre': gnrname, 'song_id': song_id, 'artist_id': artist_id, 'song_name': song_name, 'artist_name': artist_name}, ignore_index=True)
        

df.to_csv('fork_nat.csv', index=False)

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


# 포크 국악 합쳐서 전체 중복체크

In [11]:
df1 = pd.read_csv('fork_nat.csv')
df2 = pd.read_csv('new_new.csv')
# append df1 to df2
df = df1.append(df2)
df = df.drop_duplicates(subset=['song_id'])
df = df.reset_index(drop=True)
df.to_csv('data_0727.csv', index=False)

# print value count by genre
df['genre'].value_counts()

발라드    559
힙합     486
댄스     448
트로트    435
포크     433
국악     418
Name: genre, dtype: int64

# 최종 데이터에 'lyric' 컬럼 추가

In [14]:
data = pd.read_csv('data_0727.csv')
data.head()

# add 'lyric' column
data['lyric'] = ''
data.head()
# save
data.to_csv('data_0727.csv', index=False)

# 가사 가져오기

In [30]:
data = pd.read_csv('songs_fin.csv')
data.head()

for i in range(len(data)):
  if pd.isna(data.iloc[i, 5]): # if lyric is nan
    sleepTime = random.randint(1, 4)
    time.sleep(sleepTime)

    song_id = data['song_id'][i]
    url = 'https://www.melon.com/song/detail.htm?songId=' + str(song_id)
    r = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    html = r.text
    soup = BeautifulSoup(html, 'html.parser')
    if '페이지를 찾을 수 없습니다' in soup.text:
      print('error')
      break
    else:

      lyric = soup.select_one('#d_video_summary')
      lyric = str(lyric).split('-->')[-1]
      lyric = lyric.split('</div>')[0]
      lyric = lyric.strip()
      lyric = lyric.replace('<br/>', ' ')
      
      data.iloc[i, 5] = lyric

      # save df to csv
      data.to_csv('songs_fin.csv', index=False, encoding='utf-8')